In [1]:
# Import necessary libraries
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, pipeline, Trainer, TrainingArguments
from torch.utils.data import DataLoader
import torch
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification

# Load a dataset from the Hugging Face Hub
dataset_name = "imdb"
dataset = load_dataset(dataset_name, split="train[:1%]")  # Load a subset (10%) of the training data to reduce computational resources

# Define the model and tokenizer name
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

# Load the pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Set the format to return PyTorch tensors
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Prepare the dataset for PyTorch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_loader = DataLoader(tokenized_dataset, batch_size=8, collate_fn=data_collator)

# Load the pretrained model
base_model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Print the model architecture before fine-tuning
print("Model architecture before fine-tuning:")
print(base_model)

# Load the metric
metric = load_metric("accuracy")

# Function to compute metrics
def compute_metrics(predictions, references):
    return metric.compute(predictions=predictions, references=references)

# Evaluate the base model on the dataset
print("\nEvaluating the base model...")
base_predictions = []
base_references = []

for batch in data_loader:
    inputs = {key: value for key, value in batch.items() if key != 'text'}
    with torch.no_grad():
        outputs = base_model(**inputs)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=-1).numpy()
    labels = inputs['labels'].numpy()
    base_predictions.extend(preds)
    base_references.extend(labels)

# Compute accuracy for the base model
base_accuracy = compute_metrics(base_predictions, base_references)

# Print the accuracy of the base model
print(f"\nBase Model Accuracy: {base_accuracy['accuracy']:.4f}")

# Initialize PEFT configuration and model
peft_config = LoraConfig(
    r=4,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["attention.q_lin", "attention.v_lin"]  # Adjusted for DistilBERT architecture
)

peft_model = get_peft_model(base_model, peft_config)

# Print the model architecture after applying PEFT
#print("\nModel architecture after applying PEFT:")
#print(peft_model)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Initialize the Trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=lambda p: compute_metrics(p.predictions.argmax(-1), p.label_ids)
)

# Fine-tune the model
trainer.train()

# Save the PEFT model
peft_model.save_pretrained('./peft_model')

# Load the saved PEFT model for inference
loaded_peft_model = AutoPeftModelForSequenceClassification.from_pretrained('./peft_model')

# Evaluate the PEFT model on the dataset after fine-tuning
print("\nEvaluating the PEFT model after fine-tuning...")
peft_predictions = []
peft_references = []

for batch in data_loader:
    inputs = {key: value for key, value in batch.items() if key != 'text'}
    with torch.no_grad():
        outputs = loaded_peft_model(**inputs)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=-1).numpy()
    labels = inputs['labels'].numpy()
    peft_predictions.extend(preds)
    peft_references.extend(labels)

# Compute accuracy for the PEFT model
peft_accuracy = compute_metrics(peft_predictions, peft_references)

# Print the accuracy of the PEFT model after fine-tuning
print(f"\nPEFT Model Accuracy after fine-tuning: {peft_accuracy['accuracy']:.4f}")

# Test the classifier with an example sentence using the loaded PEFT model
example_sentence = "I love using Hugging Face models for NLP tasks!"
classifier = pipeline("sentiment-analysis", model=loaded_peft_model, tokenizer=tokenizer)
result = classifier(example_sentence)

# Print the result
print("\nSentiment analysis result for the example sentence:")
print(result)


Model architecture before fine-tuning:
DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dr

/tmp/ipykernel_3076/2507392923.py:39: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")



Base Model Accuracy: 0.9240

Model architecture after applying PEFT:
PeftModel(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
        

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,No log
2,No log,No log
3,No log,No log


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Evaluating the PEFT model after fine-tuning...


The model 'PeftModelForSequenceClassification' is not supported for sentiment-analysis. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSequenceClassification',


PEFT Model Accuracy after fine-tuning: 0.9680

Sentiment analysis result for the example sentence:
[{'label': 'POSITIVE', 'score': 0.9960821270942688}]
